## Grouped Aggregations

Let us go through the details related to aggregation using Spark.

* We can perform total aggregations directly on Dataframe or we can perform aggregations after grouping by a key(s).
* Here are the APIs which we typically use to group the data using a key.
  * groupBy
  * rollup
  * cube
* Here are the functions which we typically use to perform aggregations.
  * count
  * sum, avg
  * min, max
* If we want to provide aliases to the aggregated fields then we have to use `agg` after `groupBy`.
* Let us get the count of flights for each day for the month of 200801.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [2]:
airtraffic_path = "/public/airtraffic_all/airtraffic-part/flightmonth=200801"

In [3]:
airtraffic = spark. \
    read. \
    parquet(airtraffic_path)

In [4]:
from pyspark.sql.functions import concat, lpad, count, lit

In [5]:
airtraffic. \
    groupBy(concat("year",
                   lpad("Month", 2, "0"),
                   lpad("DayOfMonth", 2, "0")
                  ).alias("FlightDate")
           ). \
    agg(count(lit(1)).alias("FlightCount")). \
    show()

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|  20080120|      18653|
|  20080130|      19766|
|  20080115|      19503|
|  20080118|      20347|
|  20080122|      19504|
|  20080104|      20929|
|  20080125|      20313|
|  20080102|      20953|
|  20080105|      18066|
|  20080111|      20349|
|  20080109|      19820|
|  20080127|      18903|
|  20080101|      19175|
|  20080128|      20147|
|  20080119|      16249|
|  20080106|      19893|
|  20080123|      19769|
|  20080117|      20273|
|  20080116|      19764|
|  20080112|      16572|
+----------+-----------+
only showing top 20 rows



* Using order_items, get revenue for each order.

In [7]:
order_items_path = '/public/retail_db/order_items'

In [8]:
order_items = spark. \
    read. \
    csv(order_items_path, 
        schema="""
            order_item_id INT, order_item_order_id INT,
            order_item_product_id INT, order_item_quantity INT,
            order_item_subtotal FLOAT, order_item_product_price FLOAT
        """
       )

In [9]:
from pyspark.sql.functions import sum

In [10]:
order_items. \
    groupBy('order_item_order_id'). \
    agg(sum('order_item_subtotal').alias('revenue_per_order')). \
    show()

+-------------------+------------------+
|order_item_order_id| revenue_per_order|
+-------------------+------------------+
|              61793|1299.8700218200684|
|              62015|1139.8800354003906|
|              62680| 379.9600067138672|
|              62985|249.89999389648438|
|              63087|349.96001052856445|
|              63106| 1299.910026550293|
|              63155| 909.9200134277344|
|              63271| 749.9800109863281|
|              63574|199.99000549316406|
|              63645| 1269.900032043457|
|              63964|359.97000885009766|
|              64121|229.99000549316406|
|              64519|1179.9400253295898|
|              64628| 869.9100112915039|
|              64859| 399.9700012207031|
|              65220| 749.9800109863281|
|              65241| 1099.910026550293|
|              65251|1149.9300231933594|
|              65408| 598.9800109863281|
|              65867| 909.8900260925293|
+-------------------+------------------+
only showing top

In [12]:
from pyspark.sql.functions import round

In [13]:
order_items. \
    groupBy('order_item_order_id'). \
    agg(round(sum('order_item_subtotal'), 2).alias('revenue_per_order')). \
    show()

+-------------------+-----------------+
|order_item_order_id|revenue_per_order|
+-------------------+-----------------+
|                148|           479.99|
|                463|           829.92|
|                471|           169.98|
|                496|           441.95|
|               1088|           249.97|
|               1580|           299.95|
|               1591|           439.86|
|               1645|          1509.79|
|               2366|           299.97|
|               2659|           724.91|
|               2866|           569.96|
|               3175|           209.97|
|               3749|           143.97|
|               3794|           299.95|
|               3918|           829.93|
|               3997|           579.95|
|               4101|           129.99|
|               4519|            79.98|
|               4818|           399.98|
|               4900|           179.97|
+-------------------+-----------------+
only showing top 20 rows



* Get min and max order_item_subtotal for each order id.

In [14]:
from pyspark.sql.functions import min, max

In [15]:
order_items. \
    groupBy('order_item_order_id'). \
    agg(
        round(sum('order_item_subtotal'), 2).alias('revenue_per_order'),
        min('order_item_subtotal').alias('order_item_subtotal_min'),
        max('order_item_subtotal').alias('order_item_subtotal_max')
    ). \
    show()

+-------------------+-----------------+-----------------------+-----------------------+
|order_item_order_id|revenue_per_order|order_item_subtotal_min|order_item_subtotal_max|
+-------------------+-----------------+-----------------------+-----------------------+
|                148|           479.99|                  100.0|                  250.0|
|                463|           829.92|                  39.99|                 299.97|
|                471|           169.98|                  39.99|                 129.99|
|                496|           441.95|                  49.98|                  150.0|
|               1088|           249.97|                 119.98|                 129.99|
|               1580|           299.95|                 299.95|                 299.95|
|               1591|           439.86|                  39.99|                 199.95|
|               1645|          1509.79|                 159.96|                 399.98|
|               2366|           